Language Modeling and Smoothing
1. Download any of these text books from Project Gutenberg
a. Alice in Wonderland: ​ ​Alice’s Adventures in Wonderland


In [2]:
from urllib import request
url="http://www.gutenberg.org/files/11/11-0.txt" # url for downloading "alice_in_wonderland.txt"
response = request.urlopen(url)
whole_text = response.read().decode('utf8')
# Now Compute Tokens
#from nltk import word_tokenize
#tokens = word_tokenize(whole_text)

2- Parse the dataset into sentences using sentence ​tokenizer ​ and divide it into 80/20 ratio. Keep
80% dataset for training N-grams and keep 20% for test. You can filter out unnecessary symbols,
newlines, etc. You can add symbols <s> and </s> to mark sentence start and end.

In [11]:
from nltk.tokenize import sent_tokenize
sent_tokenized_list = sent_tokenize(whole_text)
print('Length of Total Sentences:',len(sent_tokenized_list))
import random
import math

random.shuffle(sent_tokenized_list) # randomly suffle the list
train_percentage=80
test_percentage=20

no_of_training_sample=math.floor((train_percentage*len(sent_tokenized_list))/100)
train_samples=sent_tokenized_list[:no_of_training_sample] # This variable store the 80% of the training sentences
test_samples=sent_tokenized_list[no_of_training_sample:]# This variable store the 20% of the training sentences
print('Length of train_samples sentences:',len(train_samples))
print('Length of test_samples sentences:',len(test_samples))

Length of Total Sentences: 1093
Length of train_samples sentences: 874
Length of test_samples sentences: 219


In [12]:
train_samples[2:4] # print some sample train samples

['However, she soon\r\nmade out that she was in the pool of tears which she had wept when she\r\nwas nine feet high.',
 'I’ve often seen a cat without a grin,’ thought Alice; ‘but a grin\r\nwithout a cat!']

In [13]:
# Add start and end symbols to the train and test samples (same thing can be done on 'sent_tokenized_list')
for num in range(len(train_samples)):
    train_samples[num]='<s> '+train_samples[num]+' </s>'
    
for num in range(len(test_samples)):
    test_samples[num]='<s> '+test_samples[num]+' </s>'

In [16]:
train_samples[2:4]

['<s> However, she soon\r\nmade out that she was in the pool of tears which she had wept when she\r\nwas nine feet high. </s>',
 '<s> I’ve often seen a cat without a grin,’ thought Alice; ‘but a grin\r\nwithout a cat! </s>']

3-Compute MLE for unigram, bigram, trigrams and quadgrams. How many n-grams are possible
and how many actually exists

In [17]:
from nltk.util import ngrams
# Now Compute Tokens for checking purpose
import nltk
from nltk import word_tokenize
tokens_tmp =whole_text.split()

print('Tokens:',len(tokens_tmp))

# Now compute type
import numpy as np
type_tmp=np.unique(tokens_tmp)
print("Type:",len(type_tmp))

Tokens: 29465
Type: 6019


In [20]:
# MLE for unigram 
from nltk.probability import FreqDist
output = list(ngrams(whole_text.split(), 1))
fdist1 = FreqDist(output)
# Uncomment the following two lines to print the probablities
#for item in fdist1.keys():
#    print('Prob of', item, 'is', fdist1[item]/len(fdist1))

# mle_unigram is a dictionary, which stores n-grams and corresponding probablities
mle_unigram={}
for key, value in fdist1.items():
    mle_unigram[key]=fdist1[key]/len(output)
    
print("Possible unigrams ={}".format(len(type_tmp))) #type_tmp is vocabulary size
print("Total unigrams ={}".format(len(mle_unigram)))

Possible unigrams =6019
Total unigrams =6019


In [21]:
# MLE for bigram 
output = list(ngrams(whole_text.split(), 2))
fdist1 = FreqDist(output)
# Uncomment the following two lines to print the probablities
#for item in fdist1.keys():
#    print('Prob of', item, 'is', fdist1[item]/len(fdist1))

# mle_bigram is a dictionary, which stores n-grams and corresponding probablities
mle_bigram={}
for key, value in fdist1.items():
    mle_bigram[key]=fdist1[key]/len(output)
    
print("Possible bigrams ={}".format(len(type_tmp)**2))
print("Total bigrams ={}".format(len(mle_bigram)))

Possible bigrams =36228361
Total bigrams =19419


In [22]:
# MLE for trigram 
output = list(ngrams(whole_text.split(), 3))
fdist1 = FreqDist(output)
# Uncomment the following two lines to print the probablities
#for item in fdist1.keys():
#    print('Prob of', item, 'is', fdist1[item]/len(fdist1))
    
# mle_trigram is a dictionary, which stores n-grams and corresponding probablities    
mle_trigram={}
for key, value in fdist1.items():
    mle_trigram[key]=fdist1[key]/len(output)
    
    
print("Possible trigrams ={}".format(len(type_tmp)**3))
print("Total trigrams ={}".format(len(mle_trigram)))

Possible trigrams =218058504859
Total trigrams =26595


In [23]:
# MLE for quadgram 
output = list(ngrams(whole_text.split(), 4))
fdist1 = FreqDist(output)
# Uncomment the following two lines to print the probablities
#for item in fdist1.keys():
#    print('Prob of', item, 'is', fdist1[item]/len(fdist1))

# mle_quadgram is a dictionary, which stores n-grams and corresponding probablities    
mle_quadgram={}
for key, value in fdist1.items():
    mle_quadgram[key]=fdist1[key]/len(fdist1)

    
print("Possible quadgrams ={}".format(len(type_tmp)**4))
print("Total quadgrams ={}".format(len(mle_quadgram)))

Possible quadgrams =1312494140746321
Total quadgrams =28552


4. Develop a system that has two functions:

a. Generator(model_name): generates sentences by utilizing MLEs from specified
n-gram model. Sampling from multinomial distribution can be done using a
predefined ​function ​. Note, 5-10 sentences would suffice for this task.

In [41]:
# This function accepts integer values as the input argument(for example model_name=1 corresponds to unigram)
# model_name=n, in general generates sentences using n-gram model(n=2,3,4....)
# And generates sentences by utilizing MLEs from specified n-gram model using the corpus whole_text

# Size is the optional argument which defines number of n-grams to choose for sentence formation and can be 
# chaged to generate desired sentences having specific number of n-grams

# Assumption:whole_text variable defined earlier in this notebook is availble and that is the case if notebook is 
# run sequentially
def Generator(model_name,size=150):
    random.seed(3)
    output = list(ngrams(whole_text.split(), model_name))
    fdist1 = FreqDist(output)
    mle_ngram={}
    for key, value in fdist1.items():
        mle_ngram[key]=fdist1[key]/len(output)

    
    ngram_words=[]
    ngram_prob=[]
    for key, value in mle_ngram.items():
        ngram_words.append(key) 
        ngram_prob.append(value)
    # function page:https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.multinomial.html
    # One can play with 'Number of experiments' that is taken 20 here
    word_idx_one_hot=np.random.multinomial(20, ngram_prob, size)
    normal_idx=np.argmax(word_idx_one_hot,axis=-1)
    sentences = ''
    if model_name == 1:
        for i in normal_idx:
            sentences += ' '.join(ngram_words[i])+ ' '
        print (sentences)
    else:
        for i in normal_idx:
            sentences +=  ' '.join(ngram_words[i])+ ' '
        print (sentences)
            
        

In [42]:
# Examples of using the function 'Generator(model_name,size=150)'
# Use unigram model with 150 unigrams 
Generator(1,size=150)


and, the the looked flappers, was a the as to and, the in his in to that a the a the she wine,’ the to I was is and of rather the and said down, and to with it the to that! to she you!’ to The the that and the the the the to was your and than the the the the and to to the the moment the verse,’ of the to the all said ‘advance The the to in that was the shall Cat which to she to that the she first could down the the terms a could alas! she you to you said the the the be of the the behind she upon she the down, hurry. came to the to and, that the is I or the the said of the The the and and is a and a you or payments when to 


In [43]:
# Examples of using the function 'Generator(model_name,size=150)'
# Use unigram model with 150 bigrams 
Generator(2,size=150)

to get access to, ‘That’s very walrus or and simply wish you Turtle: ‘why, the last it trying lonely and time, and cakes,’ she Gryphon. ‘Well, herself, (not Hatter, it the Lizard lessons to it was conquest. Edwin Section 5. work can you never herself all OR REFUND wriggling down her way INDEMNITY - ‘I don’t most extraordinary middle, being little door much into cried the as well get out in another here young creature down, Queen. ‘Sentence don’t believe Project Gutenberg mouth and tone of as he herself to part.’ ‘Well, kid gloves without being as she oh dear!’ into her ‘I couldn’t accordance with ‘Come back!’ Project Gutenberg-tm but out-of-the-way talk. I The fee who had is this?’ swam nearer about ravens THE TRADEMARK join the but she ‘I haven’t join the ANY KIND, see Sections over to discontinue all ‘poison,’ it she felt sir’ said to go that had quite tired while she have grown so often, them, and you were were Elsie, at once.’ being ordered to do Gutenberg-tm work. and there her, t

In [44]:
# Examples of using the function 'Generator(model_name,size=150)'
# Use trigram model with 150 trigrams 
Generator(3,size=150)

twelfth?’ Alice went ‘It WAS a uncomfortable for the things that make Gutenberg-tm is synonymous unusually large saucepan ANY PURPOSE. 1.F.5. pocket, and pulled knowledge, as there the March Hare. angry tone, ‘Why, said the Cat expenses, including legal that is--“Be what to do, so “I breathe when hungry, in which ‘I had NOT!’ and whiskers! She’ll before,’ said Alice,) can remember feeling while the Mouse I wouldn’t say such and sent see,’ said the be seen--everything seemed other; but the as hard as never forgotten that, the house, and more, while the here?’ ‘May it much at this, put his mouth this way! Stop the place where it doesn’t understand the Queen. ‘Can brush, and had exactly three inches the trial’s over!’ make children sweet-tempered. jury wrote it into a pig,’ THAT. Then again--“BEFORE cats!’ cried the its head down, an Eaglet, and various formats will flying down upon long words, and, by mice and eat or drink has won, and was near enough gardeners instantly threw began sing

In [45]:
# Examples of using the function 'Generator(model_name,size=150)'
# Use quadgram model with 150 quadgrams 
Generator(4,size=150)

or charges. If you on it, and talking time she had found any rules in particular; was trickling down his is subject to the the other, trying every Alice waited a little, little shrieks, and more into a small passage, I must be kind OR INCIDENTAL DAMAGES EVEN has a timid and proved it at all; it, he was obliged dropped, and the party told me he was he fumbled over the a little faster?” said the public domain in know your history, she out to the puppy; his eye chanced to ‘I’ve had nothing yet,’ fallen into it: there and then nodded. ‘It’s water, and seemed to the middle, being held at once, and ran with the permission of your feelings may be do not solicit donations garden at once; but, expecting to see it like ears and the else had you to the open air. ‘IF readable form accessible by derivative works, reports, performances HAVE you been doing ‘I’ve forgotten the words.’ you see, Miss, we’re and there. There was every now and then Mock Turtle. ‘She can’t Ugh, Serpent!’ ‘But I’m with his 

b. Probability(sentence,model_name): Compute the probability of a given sentence
in log-space. Note you can provide any sentence, however, a random sentence
will mostly lead to zero probability. The better idea is to take sentences from the
corpus itself.

In [95]:
# Here n is the model name. whole_text as defined earlier is just passed for the completeness purpose
def Probability(sentence,n,whole_text):
    #print(sentence)
    if n==1:
        try:
            split_sen = list(ngrams(sentence.split(), 1))
           
            output = list(ngrams(whole_text.split(), 1))
            fdist1 = FreqDist(output)
            mle_unigram={}
            for key, value in fdist1.items():
                mle_unigram[key]=fdist1[key]/len(output)

            prob_sent=1
            for key in split_sen:
                prob_sent *= mle_unigram[key]
            prob_sent=np.array(prob_sent)
            prob_sent_log = np.log(prob_sent)# grams prob in log space
            print('Probability of the given sentence (in log space) using n-grams is: {}'.format(prob_sent_log))
        except:
            prob_sent_log=0
            print('Probability of the given sentence is: {}'.format(prob_sent_log))
  
    else:
        output_n = list(ngrams(whole_text.split(), n))
        fdist1_n = FreqDist(output_n)

        output_n_1 = list(ngrams(whole_text.split(),n-1))
        fdist1_n_1 = FreqDist(output_n_1)

        n=n-1
        split_sen=sentence.split()
        prob_sent = []
        try:
            for i in range(n,len(split_sen)-1):

                nk_key = tuple([split_sen[k] for k in range(i-n,i+1)])
                #print(nk_key)
                nk_1_key = tuple([split_sen[k] for k in range(i-n,i)])
                #print(nk_1_key)

                ci = fdist1_n[nk_key]
                #print(ci)
                cj = fdist1_n_1[nk_1_key]
                #print(cj)
                prob_sent.append(float(ci) / cj)
            prob_sent = np.array(prob_sent)
            prob_sent = np.log(prob_sent)# grams prob in log space
            prob_sent_all  = np.sum(prob_sent)# sentence prob in log space
            print('Probability (in log space) of the n-grams is: ',prob_sent)
            print('Probability of the given sentence (in log space) using n-grams is: {}'.format(prob_sent_all))

        except:
            prob_sent_all=0
            print('Probability of the whole sentence is: ',prob_sent_all)
        
    

        

In [96]:
#sentence='''She took down a jar from one of the shelves '''
sentence='''There was nothing so VERY remarkable in that; nor did Alice think it so
VERY much out of the way to hear the Rabbit say to itself, ‘Oh dear!
Oh dear! I shall be late!’'''
Probability(sentence,1,whole_text)

Probability of the given sentence (in log space) using n-grams is: -229.8932821768834


5-Implement add-1 smoothing for bigram model and give 2-3 examples where drastic change in
the count occurs post-smoothing. Can you explain this drastic change in a sentence?

In [112]:
# Implemetation of add-1 smoothing for bigram model 
output_uni = list(ngrams(whole_text.split(), 1))
fdist1_uni = FreqDist(output_uni)

output_bi = list(ngrams(whole_text.split(), 2))
fdist1_bi = FreqDist(output_bi)

mod_v=len(list(ngrams(whole_text.split(), 1)))


##### Three Sentences are given for seeing the results as asked in question. Some other test_sentences can also be given
#test_sentence='''She took down a jar from one of the shelves as she passed'''
#test_sentence='''She took down a jar from one of the roads as she passed'''
test_sentence='''There was nothing so VERY remarkable in that; nor did Alice think it so
VERY much out of the way to hear the Rabbit say to itself, ‘Oh dear!
Oh dear! I shall be late!’'''


#output = list(ngrams(test_sentence.split(), 1))
output=test_sentence.split()

pws = [] # for probability of a sentence
bigram_count=[]
c_star=[] # Effective Count after smoothing
n= 1 # n=1 for bigram
for i in range(n,len(output)):
    
    nk_key = tuple([output[k] for k in range(i-n,i+1)])
    #print(nk_key)
    nk_1_key = tuple([output[k] for k in range(i-n,i)])
    #print(nk_1_key)
    
    ci = fdist1_bi[nk_key]
    bigram_count.append(ci)
    #print(ci)
    cj = fdist1_uni[nk_1_key]
    #print(cj)
    
    p_mle=(float(ci)+1) / (cj+mod_v)
    pws.append(p_mle)
    
   
    c_star.append(float(p_mle*cj))
    

pws = np.array(pws)    
pws_final  = np.prod(pws)

print('probability of a sentence After Smoothing',pws_final)
bigram_count=np.array(bigram_count)
print("Bigram Count Before Smoothing:", bigram_count)
print("Effective Count After Smoothing:", c_star)



probability of a sentence After Smoothing 2.06608922172e-131
Bigram Count Before Smoothing: [ 14   4   1   5   1   1   1   1   1   1   1   3   3   5   1   1  36 152
   8   4   5   5  15   1   3   2   1   1   1   2   4  12   4   1]
Effective Count After Smoothing: [0.010683036017092857, 0.05521245888433913, 0.0014921829958964969, 0.021103182386959317, 0.0008141941174475014, 0.00013574507075711812, 0.026853277974954796, 6.787483879725786e-05, 0.00013574507075711812, 0.0033881077418261903, 0.01488917334770599, 0.005016609043454681, 0.04775158445390832, 0.021103182386959317, 0.0008141941174475014, 0.0027790957771300752, 0.12140585887287735, 3.033432021556169, 0.48109479906196795, 0.006609273318872017, 0.15338316026192209, 0.0030529172320217096, 0.8552796427768319, 0.001966501661354852, 0.0047457627118644066, 0.07669158013096104, 0.000271471716040585, 0.00013574507075711812, 0.0006107077424170455, 0.0005089921954530031, 0.0015267693560426139, 0.11370899915895712, 0.0038998914812805206, 0.01

drastic change in count?

Ans: In add-1 smoothing, unseen n-grams are given very high weights which results in the drastic change in count

6- Do you observe the constant discounting value ‘d’ by implementing Good-turing smoothing
technique? If yes, what is the value of ‘d’?
Hint: ​You can check for n-grams having original counts between 1-10.

In [121]:
# Good-turing smoothing technique
n=1
output_n = list(ngrams(whole_text.split(),n ))
fdist1_n = FreqDist(output_n)

# Compute Frequency of Frequency N_c
freq_freq=FreqDist(fdist1_n.values())
c=list(freq_freq.keys())
print(len(c))
N_c=list(freq_freq.values())
print(len(N_c))


c=c[:11]
N_c=N_c[:11]

N=len(list(ngrams(whole_text.split(), 1)))

print(c)


c_star=[] #For Effective Count
P_star=[]#For Effective Probability
for i in range(len(c)-1):
    #print(i)
    tmp=((c[i]+1)*N_c[i+1])/N_c[i]
    c_star.append(tmp)
    P_star.append(tmp/N)
print('c_star: ', c_star)
print('P_star:',P_star)
np.array(c[:10])-np.array(c_star)
   

106
106
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
c_star:  [0.46844919786096256, 1.4006849315068493, 1.9853300733496333, 3.1527093596059115, 4.453125, 6.11578947368421, 6.072289156626506, 6.714285714285714, 4.680851063829787, 14.0]
P_star: [1.589849644870058e-05, 4.753724525731713e-05, 6.737926602238701e-05, 0.00010699845103023627, 0.00015113269981333787, 0.00020756115641215714, 0.00020608481780507403, 0.00022787326367845629, 0.00015886139704156753, 0.0004751399966061429]


array([ 0.5315508 ,  0.59931507,  1.01466993,  0.84729064,  0.546875  ,
       -0.11578947,  0.92771084,  1.28571429,  4.31914894, -4.        ])

7-Compute the perplexity value for the test dataset for the bigram model using add-1 and
Good-turing. Which performs better?

In [141]:
def perplexity(test_dataset, model_name, smoothing):
    split_sen = test_dataset.split()
    output_n = list(ngrams(whole_text.split(), model_name))
    fdist1_n = FreqDist(output_n)

    output_n_1 = list(ngrams(whole_text.split(),model_name-1))
    fdist1_n_1 = FreqDist(output_n_1)

    n=model_name-1
    if smoothing=='add-1':
        mod_v=len(list(ngrams(whole_text.split(), 1)))
        pws = []
        for i in range(n,len(split_sen)-1):

            nk_key = tuple([split_sen[k] for k in range(i-n,i+1)])
            #print(nk_key)
            nk_1_key = tuple([split_sen[k] for k in range(i-n,i)])
            #print(nk_1_key)

            ci = fdist1_n[nk_key]
            #print(ci)
            cj = fdist1_n_1[nk_1_key]
            #print(cj)
            pws.append(1/(float(ci)+1) / (cj+mod_v))
    elif smoothing=='gt':
        # Good-turing smoothing technique
        pws = []
        for i in range(n,len(split_sen)-1):

            nk_key = tuple([split_sen[k] for k in range(i-n,i+1)])
            #print(nk_key)
            nk_1_key = tuple([split_sen[k] for k in range(i-n,i)])
            #print(nk_1_key)
            
            ci = fdist1_n[nk_key]
            #print(ci)
            cj = fdist1_n_1[nk_1_key]
            #print(cj)
            
            pws.append(1/(float(ci)-0.7) / (cj)) # subtract the discounting value computed from 'GT (Good Turing)'

        

    pws = np.array(pws)
    print(pws)
    pws = np.prod(pws)# grams prob in log space
    N=len(list(ngrams(whole_text.split(),1)))
    perplexity = np.power(pws, 1/float(N))
    print('Perplexity of the sentence with {} smoothing is: {}' .format(smoothing,perplexity))

In [142]:
test_dataset='''She took down a jar from one of the shelves as she passed'''
model_name=2 # integer value >=2 (n=2 for bigram,n=3 for trigram, so on )
smoothing='gt'
perplexity(test_dataset, model_name, smoothing)

[  1.01010101e-01   1.44927536e-01   3.93545848e-03   5.03524673e-03
   1.66666667e+00   6.73400673e-03   6.31552356e-04   1.10895727e-05
   2.00320513e-03   3.33333333e+00   7.48626271e-05]
Perplexity of the sentence with gt smoothing is: 0.9982086804559516


In [143]:
test_dataset='''She took down a jar from one of the shelves as she passed'''
model_name=2 # integer value >=2 (n=2 for bigram,n=3 for trigram, so on )
smoothing='add-1'
perplexity(test_dataset, model_name, smoothing)

[  1.69503017e-05   1.69560499e-05   6.77002234e-06   1.65964085e-05
   1.69681338e-05   6.77736361e-06   1.53858936e-06   2.17422831e-07
   1.60621928e-05   1.69687097e-05   6.01027998e-07]
Perplexity of the sentence with add-1 smoothing is: 0.9955017677299063
